# Rent Analysis 

In [2]:
import pandas as pd
import geopandas as gpd

import requests
from bs4 import BeautifulSoup

import time
import random

import re

import matplotlib.pyplot as plt
import contextily as ctx

#### Functions for cleaning unit_type column and rents column to lists

In [3]:
def clean_units(string):
    
    #drop tags 
    notags = re.sub(r"<[^>]*>", "", string)

    #drop []
    nobracket = notags.replace("[", "")
    nobrackets = nobracket.replace("]", "")
    
    #drop spaces 
    nospace = nobrackets.replace(" ", "")
    
    #split by comma 
    unitlist = nospace.split(',')
    
    #return list 
    return unitlist

In [4]:
def clean_rents(string): 
    #drop tags 
    notags = re.sub(r"<[^>]*>", "", string)

    #drop white spaces 
    nowhitespace = re.sub(r"\s+", " ", notags)
    
    #drop []
    nobracket = nowhitespace.replace("[", "")
    nobrackets = nobracket.replace("]", "")
    
    #split on " , " 
    rentlist = nobrackets.split(" , ")
    
    return rentlist

In [48]:
# saving the gdfs of 500 scraped posts for each neighborhood to csvs so I don't have to scrape again: 
eho_500 = pd.read_csv('C:/Users/audre/OneDrive/Documents/229-GitHub/Final229_csvs/eho_500_gdf.csv')
bohe_500 = pd.read_csv('C:/Users/audre/OneDrive/Documents/229-GitHub/Final229_csvs/bohe_500_gdf.csv')
ktown_500 = pd.read_csv('C:/Users/audre/OneDrive/Documents/229-GitHub/Final229_csvs/ktown_500_gdf.csv')
hipa_500 = pd.read_csv('C:/Users/audre/OneDrive/Documents/229-GitHub/Final229_csvs/hipa_500_gdf.csv')


## For Each Neighborhood... 

In [49]:
#CHANGE THIS ONE AND RUN ALL BELOW

neighborhood = hipa_500

In [50]:
#create separate dataframe to clean rents and clean units 

neighborhood['rents'] = neighborhood['rents'].apply(clean_rents)
neighborhood['unit_type'] =  neighborhood['unit_type'].apply(clean_units)
neighborhood.head(1)

,Unnamed: 0,Listing,Address,Latitude,Longitude,Description,Price,Unit Type,BR,unit_type,rents,Unique Amenities,Apartment Features,Community Amenities,URL,geometry
0,0,3821 W Avenue 43,"3821 W Avenue 43, Los Angeles, CA 90041",34.12331,-118.224,"<section class=""descriptionSection js-viewAnal...","$2,700 - 3,100",1-2 Bed,"[<div class=""priceBedRangeInfo"">\n<h3 class=""m...","[OneBedroomLofts, TwoBedroomLofts, OneBedroomL...","[ $2,700 – $2,900, $3,050 – $3,100, $2,700 – $...",[],"[<span class=""amenity"">Washer/Dryer</span>, <s...","[<ul class=""combinedAmenitiesList"">\n<li class...",https://www.apartments.com/3821-w-avenue-43-lo...,POINT (-118.224 34.12331)


Now the 'unit_type' and 'rents' columns are each lists of unit type or rent range. 

#### Clean rows with different lengths of lists 

There are rows that have different lengths for lists. 

In [51]:
# Adam's code 
problem_i = [ix for ix, row in neighborhood.iterrows() if len(row['rents']) !=len(row['unit_type'])]

In [52]:
# print the problem rows' rent columns 
for i in problem_i: 
    print(neighborhood.loc[i]['rents'])

In [53]:
for i in problem_i: 
    print(len(neighborhood.loc[i]['unit_type']) - len(neighborhood.loc[i]['rents']))

Need to add an if statement that skips rows that have ' Call for Rent' as the first item of the 'rents' list. 

if the length of unit_type =/= length of rents, then add " Call for Rent" in the remaining two or four spaces in the 'rents' list. 

In [54]:
####UNIQUE TO EACH DATAFRAME?
#### OR FUNCTIONIFY? -- drop these rows! 

neighborhood_ = neighborhood.drop(problem_i)


#####East Hollywood
# for i in problem_i:
#     if len(eho_500.loc[i]['unit_type']) - len(eho_500.loc[i]['rents']) == 2:
#         eho_500.loc[i]['rents'] == eho_500.loc[i]['rents'].extend([' Call for Rent', ' Call for Rent'])
#     if len(eho_500.loc[i]['unit_type']) - len(eho_500.loc[i]['rents']) == 4: 
#         eho_500.loc[i]['rents'] == eho_500.loc[i]['rents'].extend([' Call for Rent', ' Call for Rent',' Call for Rent', ' Call for Rent'])
#     else:
#         continue 


#### Explode in two parts

In [55]:
rentsDf = neighborhood_.explode('rents').reset_index()
unitsDf = neighborhood_.explode('unit_type').reset_index()
newDf = unitsDf 
newDf['rents_cleaned'] = rentsDf['rents']
newDf = newDf.drop(columns = 'rents')
newDf.head(2)

,index,Unnamed: 0,Listing,Address,Latitude,Longitude,Description,Price,Unit Type,BR,unit_type,Unique Amenities,Apartment Features,Community Amenities,URL,geometry,rents_cleaned
0,0,0,3821 W Avenue 43,"3821 W Avenue 43, Los Angeles, CA 90041",34.12331,-118.224,"<section class=""descriptionSection js-viewAnal...","$2,700 - 3,100",1-2 Bed,"[<div class=""priceBedRangeInfo"">\n<h3 class=""m...",OneBedroomLofts,[],"[<span class=""amenity"">Washer/Dryer</span>, <s...","[<ul class=""combinedAmenitiesList"">\n<li class...",https://www.apartments.com/3821-w-avenue-43-lo...,POINT (-118.224 34.12331),"$2,700 – $2,900"
1,0,0,3821 W Avenue 43,"3821 W Avenue 43, Los Angeles, CA 90041",34.12331,-118.224,"<section class=""descriptionSection js-viewAnal...","$2,700 - 3,100",1-2 Bed,"[<div class=""priceBedRangeInfo"">\n<h3 class=""m...",TwoBedroomLofts,[],"[<span class=""amenity"">Washer/Dryer</span>, <s...","[<ul class=""combinedAmenitiesList"">\n<li class...",https://www.apartments.com/3821-w-avenue-43-lo...,POINT (-118.224 34.12331),"$3,050 – $3,100"


#### Separate rents_cleaned range into min_rent and max_rent columns 

In [56]:
newDf['rents_split'] = newDf['rents_cleaned'].str.split('–') # copy and paste the hyphen because it's a little longer than the keyboard one 
minmaxrents = pd.DataFrame(newDf["rents_split"].to_list(), columns=['min_rent_', 'max_rent_'])
newDf = newDf.join(minmaxrents)
newDf.head(2)

,index,Unnamed: 0,Listing,Address,Latitude,Longitude,Description,Price,Unit Type,BR,unit_type,Unique Amenities,Apartment Features,Community Amenities,URL,geometry,rents_cleaned,rents_split,min_rent_,max_rent_
0,0,0,3821 W Avenue 43,"3821 W Avenue 43, Los Angeles, CA 90041",34.12331,-118.224,"<section class=""descriptionSection js-viewAnal...","$2,700 - 3,100",1-2 Bed,"[<div class=""priceBedRangeInfo"">\n<h3 class=""m...",OneBedroomLofts,[],"[<span class=""amenity"">Washer/Dryer</span>, <s...","[<ul class=""combinedAmenitiesList"">\n<li class...",https://www.apartments.com/3821-w-avenue-43-lo...,POINT (-118.224 34.12331),"$2,700 – $2,900","[ $2,700 , $2,900]","$2,700","$2,900"
1,0,0,3821 W Avenue 43,"3821 W Avenue 43, Los Angeles, CA 90041",34.12331,-118.224,"<section class=""descriptionSection js-viewAnal...","$2,700 - 3,100",1-2 Bed,"[<div class=""priceBedRangeInfo"">\n<h3 class=""m...",TwoBedroomLofts,[],"[<span class=""amenity"">Washer/Dryer</span>, <s...","[<ul class=""combinedAmenitiesList"">\n<li class...",https://www.apartments.com/3821-w-avenue-43-lo...,POINT (-118.224 34.12331),"$3,050 – $3,100","[$3,050 , $3,100]","$3,050","$3,100"


#### Now clean the min_rent_ and max_rent_ columns 
...by dropping $ and commas and spaces and converting to int. 

In [57]:
def clean_rent_column(string):  
    if string is None: 
        return None 
    
    #remove spaces 
    nospace = string.replace(" ", "")
#     print(nospace)
    
    #remove $ 
    nopunc = nospace.replace("$", "")
    
    #remove comma 
    nocomma = nopunc.replace(",", "") 
    
    #remove / 
    noslash = nocomma.replace("/", "") 
    
    #remove words
    nowords = re.sub(r"[A-z]", "", noslash)
    
    if nowords == "":
        return None 
    
    else: 
        integer = int(nowords) 
        return integer

In [58]:
newDf['min_rent_'] = newDf['min_rent_'].apply(clean_rent_column)
newDf['max_rent_'] = newDf['max_rent_'].apply(clean_rent_column)

In [59]:
newDf.head(1)

,index,Unnamed: 0,Listing,Address,Latitude,Longitude,Description,Price,Unit Type,BR,unit_type,Unique Amenities,Apartment Features,Community Amenities,URL,geometry,rents_cleaned,rents_split,min_rent_,max_rent_
0,0,0,3821 W Avenue 43,"3821 W Avenue 43, Los Angeles, CA 90041",34.12331,-118.224,"<section class=""descriptionSection js-viewAnal...","$2,700 - 3,100",1-2 Bed,"[<div class=""priceBedRangeInfo"">\n<h3 class=""m...",OneBedroomLofts,[],"[<span class=""amenity"">Washer/Dryer</span>, <s...","[<ul class=""combinedAmenitiesList"">\n<li class...",https://www.apartments.com/3821-w-avenue-43-lo...,POINT (-118.224 34.12331),"$2,700 – $2,900","[ $2,700 , $2,900]",2700.0,2900.0


# DO NOT RUN BEYOND THIS POINT 

Ran the code above for all four neighborhoods and saved the results below: 

## Rent Summary of East Hollywood 

In [17]:
newDf.min_rent_.describe()

count    4553.000000
mean     2824.142543
std       829.796040
min      1125.000000
25%      2291.000000
50%      2815.000000
75%      3425.000000
max      5554.000000
Name: min_rent_, dtype: float64

In [18]:
newDf.max_rent_.describe()

count    2128.000000
mean     3226.000000
std       859.223175
min      1685.000000
25%      2600.000000
50%      3150.000000
75%      3926.250000
max      4850.000000
Name: max_rent_, dtype: float64

Median Rent in East Holywood is $2815-3150. 

In [20]:
newDf.groupby('unit_type')[['min_rent_', 'max_rent_']].median().head(50)

#print(joinedDf.groupby('PARTY_SOBRIETY')[['NUMBER_KILLED','NUMBER_INJURED']].mean())

,min_rent_,max_rent_
unit_type,,
#7,2245.0,NaN
.25,1650.0,NaN
10,1450.0,NaN
105,1595.0,NaN
1050,2872.0,NaN
11,1772.5,NaN
1180,NaN,NaN
12,1600.0,NaN
16,1700.0,NaN


Many of the listings seem to mark the apartment unit # instead of a BR/BD unit type - which is unfortunately not useful for getting an aggregate estimate of rents per unit type. 

Here are the rents that do seem to be in some summary form by unit type. I just took a look at the minimum links.

* 1BR: 1895
* 1Bed/1Bath: 1982
* 1Br-1B: 2355
* 1bedroom: 1970.5

The 1Br-1B seems a bit high? It probably is an outlier. The 1 bedroom with 1 bathroom unit type hover around $ 1900 a month.


* 1Bed2Bath: 2315


* 2Bed/1Bath: 2125

It makes sense that the 2 bathroom would bump up the price a bit, but why is 1 bed+1bath higher than 2bed+1bath?


* 2BD+2BTH: 2654
* 2Bed/2Bath: 2430
* 2Bed2Bath: 2430
* 2Bed/1.5Bath: 2549

The 2 bed+2 bath units are about $2,500 per month. 


* 3BED+2BTH: 3795
* 3BDSharedStudioFlat: 1619
* 3bedroom:4200 

So if each bedroom in a 3BR unit is $1,619,that would come out to $4,857 a month, a bit higher than the 4 bedroom unit, which at $1,195 per month would be $4,780.

* 4-bedshare: 1195


* 5BDSharedStudioFlat:1483


## Rent Summary of Boyle Heights

In [17]:
newDf.min_rent_.describe()

count    3382.000000
mean     2740.484033
std       812.944069
min      1175.000000
25%      2204.000000
50%      2601.000000
75%      3217.000000
max      4950.000000
Name: min_rent_, dtype: float64

In [19]:
newDf.max_rent_.describe()

count    2432.000000
mean     3588.546875
std      1020.269873
min      1650.000000
25%      2937.750000
50%      3478.000000
75%      3942.500000
max      6370.000000
Name: max_rent_, dtype: float64

Median Rent in Boyle Heights ranges from $ 2,601 - $ 3,478. 

I tried to group by unit_type, but the unit_type information really isn't there. 

In [20]:
newDf.groupby('unit_type')[['min_rent_', 'max_rent_']].median().head(50)

#print(joinedDf.groupby('PARTY_SOBRIETY')[['NUMBER_KILLED','NUMBER_INJURED']].mean())

,min_rent_,max_rent_
unit_type,,
,NaN,NaN
1.1A,NaN,NaN
1.1B,NaN,NaN
1.1C,2108.0,2551.0
1.1D,NaN,NaN
1.1E,NaN,NaN
1.1F,2062.0,2765.0
1.1G,NaN,NaN
1.1H,2369.0,2859.0


Unforunately, none of the unit-types listed appear useable.

## Rent Summary of Koreatown

In [32]:
newDf.min_rent_.describe()

count    13046.000000
mean      3187.902192
std       3154.708156
min       1395.000000
25%       2125.000000
50%       2620.000000
75%       3185.000000
max      31995.000000
Name: min_rent_, dtype: float64

In [33]:
newDf.max_rent_.describe()

count    7126.000000
mean     2934.578726
std       834.423505
min      1685.000000
25%      2245.000000
50%      2775.000000
75%      3426.250000
max      5653.000000
Name: max_rent_, dtype: float64

Median Rent in Koreatown ranges from $ 2,620 - $ 3,426.  

In [34]:
newDf.groupby('unit_type')[['min_rent_', 'max_rent_']].median().head(50)

#print(joinedDf.groupby('PARTY_SOBRIETY')[['NUMBER_KILLED','NUMBER_INJURED']].mean())

,min_rent_,max_rent_
unit_type,,
#110,2550.0,NaN
102,2500.0,NaN
103,1395.0,NaN
104,2350.0,NaN
11,2100.0,NaN
1132SHooverSt,3600.0,NaN
116,3200.0,NaN
1187CrenshawBlvd,3400.0,NaN
119,1495.0,NaN


* 1Bed/1Bath: 1815
* 1Bed1Bath: 2100
* 1Bed1Bath648Sqft: 1853
* 1Bedroom1Bath: 1975

* 1BD1Ba+Den: 2382 
* 1BD2Ba+Den: 2568

I could only get the one-bedroom information for Koreatown, but it seems to be around 1900-2000 for 1BR1B and 2400 for 2 bathrooms.

## Rent Summary of Highland Park

In [60]:
newDf.min_rent_.describe()

count    1501.000000
mean     2397.645570
std       565.712968
min      1350.000000
25%      1950.000000
50%      2395.000000
75%      2775.000000
max      4000.000000
Name: min_rent_, dtype: float64

In [61]:
newDf.max_rent_.describe()

count     570.00000
mean     2538.20000
std       719.42044
min      1400.00000
25%      1947.00000
50%      2315.00000
75%      3100.00000
max      4170.00000
Name: max_rent_, dtype: float64

Median Rent in Highland Park ranges from $ 2,315 - $ 2,395. 

In [63]:
newDf.groupby('unit_type')[['min_rent_', 'max_rent_']].median().head(50)

#print(joinedDf.groupby('PARTY_SOBRIETY')[['NUMBER_KILLED','NUMBER_INJURED']].mean())

,min_rent_,max_rent_
unit_type,,
,NaN,NaN
1,2747.5,NaN
1+1,1921.0,2295.0
1-BedroomFullyRemodeled,2400.0,NaN
17,2195.0,NaN
1Bed/1.0Bath,2350.0,NaN
1Bedroom,1675.0,1750.0
1Bedx1Bath,2475.0,NaN
2+2,2746.0,3095.0


For Highland Park, here are the min rents by unit type:
* Studio: 1350


* 1Bed/1.0Bath: 2350
* 1Bedroom: 1675
* 1Bedx1Bath: 1Bedx1Bath 
* NewlyRenovatedOneBedroom: 2000
* OnebedLoft: 2746
* OneBedroom: 2147.5 
* OneBedroomLofts: 2700
* PriorRenovatedOneBedroom: 1900


* 2Bedroom: 2150
* 2Bedx1Bath: 2850
* 2-BedroomFullyRemodeled: 3200
* 2-BedroomUnit312: 2800
* 2Bed/1.0Bath: 2599
* NewlyRenovatedTwoBedroom: 2475


* 2Bed/1.5Bath:2775


* 2Bedx2Bath: 2735
* 2Bed/2.0Bath: 2899
* 2+2: 2746
* TwoBedroom: 3150
* TwoBedroomLofts: 3050 


* 3Bed/2.0Bath: 3095
* 3Bedx2Bath: 3520

The unit_type column was less standardized than I expected. If I had more time, I would have gone back to examine the html of apartments.com to see if there is a different way of scraping a more standardized unit type.